### Sample Setup for Hw3

In [1]:
#import libraries
import pulp, numpy as np, pandas as pd
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, lpSum

### Global Variables

In [2]:
num_counties = 15 #number of counties
num_leg_districits = 5 #number of legistlative districts
pop_max = 10000 #max pop allosed in each legislative district
pop_min = 2000 #min pop allowed in each legislative district

### Setup

In [3]:
#creating a variable for each county.
counties = list(map(chr, range(97, 123)))[:num_counties]

In [4]:
#creating a variable for each district
district = list(range(0,num_leg_districits))

In [5]:
#creating a population dictionary
pop_list = [2000] * num_counties
pop = dict(zip(counties, pop_list))

In [6]:
#adjacency dictionary
adj_dict = {'a':{'b','d'} , 
            'b':{'a','e','c'},
            'c':{'b','f'},
            'd':{'a','e','g'},
            'e':{'b','f','h','d'},
            'f':{'c','e','i'},
            'g':{'d','h'},
            'h':{'g','e','i'},
            'i':{'f','h'},
            'j':{'i',"k"},
            "k":{"j","l"},
            "l":{"k","m"},
            "m":{"l","n"},
            "n":{"m","o"},
            "o":{"n"}
           }

### Creating All Possible Adjacent Pairs

In [7]:
pair_temp = []
sorted_pair_temp = []
start_county = []
end_county = []
for i in counties:
    for p in adj_dict[i]:
       pair_temp.append(i+p)
       sorted_pair_temp.append(sorted(i+p))
       start_county.append(i)
       end_county.append(p)
d = {'pair_list':pair_temp,'sorted':sorted_pair_temp,"start":start_county,"end":end_county}
df = pd.DataFrame(d)
key_list = df.sorted.drop_duplicates().keys()
pair_list = df.pair_list[key_list]
start_list = df.start[key_list]
end_list = df.end[key_list]

### Creating a dataframe of each pair within each district layer

In [8]:
#this will help built the constraint a little quicker, so doing this here and now
pair_list2  = [(p,d) for p in pair_list for d in district]
start_list2  = [(s,d) for s in start_list for d in district]
end_list2  = [(e,d) for e in end_list for d in district]
d2 = {'pair_list':pair_list2,'start':start_list2,"end":end_list2}
df2 = pd.DataFrame(d2)

### Establishing Variables

In [9]:
#establish county/district variables
county_district = LpVariable.dicts("count_district_assignment",[(c,d) for c in counties for d in district],0,1,cat="Integer")
adj_pair = LpVariable.dicts("adj_pair",[(p,d) for p in pair_list for d in district],0,1,cat="Integer")

### Objective Function

In [10]:
#setting up problem
prob = LpProblem("problem",LpMinimize)
prob += lpSum([adj_pair[i] for i in adj_pair])

### Establishing constraints

In [11]:
#making sure each county is assigned to one and only one district
for c in counties:
    prob += lpSum([county_district[(c,d)] for d in district]) == 1
    
#setting up constraints on max population in each district
for d in district:
    prob += lpSum([county_district[(i,d)] * pop[i] for i in counties]) <= pop_max 

#setting up constraints on min population in each district
for d in district:
    prob += lpSum([county_district[(i,d)] * pop[i] for i in counties]) >= pop_min 
    
#counting edges and relating to the edge variables
for i in range(len(pair_list2)):
    prob +=  county_district[start_list2[i]] - county_district[end_list2[i]] <= adj_pair[pair_list2[i]]
    
for i in range(len(pair_list2)):
    prob +=  -county_district[start_list2[i]] + county_district[end_list2[i]] <= adj_pair[pair_list2[i]]

### Solve

In [ ]:
prob.solve()

### Show the Assignment of County to Legislative District

In [ ]:
assignment_list = []
assignment_county = []
assignment_district = []
for i in county_district: 
    assignment_list.append(county_district[i].varValue)
for c in counties:
    for j in district:
        assignment_county.append(c)
        assignment_district.append(j)
d3 = {"County":assignment_county, "Legislative_District": assignment_district, "assignment_list": assignment_list}
df3 = pd.DataFrame(d3)
df3 = df3[df3['assignment_list'] == 1]

In [ ]:
df3.groupby('Legislative_District')['County'].apply(' '.join).reset_index()

### Count the Number of Cut Edges

In [ ]:
edge_total = [] 
for i in pair_list2: 
    edge_total.append(adj_pair[i].varValue/2)
print('Total of Cut Edges is {}'.format(sum(edge_total)))